In [24]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

# Site URL
url="https://en.wikipedia.org/wiki/Hanoi"
# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text

# Parse HTML code for the entire site
soup = BeautifulSoup(html_content, "lxml")
#print(soup.prettify()) # print the parsed data of html

In [25]:
a = soup.find_all("table", class_="wikitable")
print("Number of tables on site: ",len(a))

Number of tables on site:  3


In [26]:
a1 = soup.findAll("table", class_="wikitable")[2]
table_rows = a1.find_all("tr")

In [31]:
head = table_rows[1]
headings = []
for item in head.find_all("th"):
    item = (item.text).rstrip("\n")
    headings.append(item)

In [32]:
headings

['Provincial Cities/Districts[30]',
 'Wards[30]',
 'Area (km2)[30]',
 'Population (2017)[30]']

In [33]:
demographics = []
for tr in table_rows:
    district_demo = tr.find_all("td")
    row_demo = [tr.text for tr in district_demo]
    demographics.append(row_demo)

In [35]:
demographics2 = pd.DataFrame(data=demographics,columns=headings)

In [36]:
demographics3 = demographics2.loc[3:14].rename(columns={"Provincial Cities/Districts[30]":"Provincial Cities/Districts","Wards[30]":"Wards","Area (km2)[30]":"Area (km2)","Population (2017)[30]":"Population (2017)"}).reset_index(drop=True)

In [37]:
demographics3

,Provincial Cities/Districts,Wards,Area (km2),Population (2017)
0,Ba Đình,14,9.224,"247,100\n"
1,Bắc Từ Liêm,13,43.35,"333,300\n"
2,Cầu Giấy,8,12.04,"266,800\n"
3,Đống Đa,21,9.96,"420,900\n"
4,Hai Bà Trưng,20,10.09,"318,000\n"
5,Hà ĐôngHT,17,47.917,"319,800\n"
6,Hoàn Kiếm,18,5.29,"160,600\n"
7,Hoàng Mai,14,41.04,"411,500\n"
8,Long Biên,14,60.38,"291,900\n"
9,Nam Từ Liêm,10,32.27,"236,700\n"


In [38]:
demographics3['Area (km2)'] = demographics3['Area (km2)'].apply(lambda x: x.replace('.',''))
demographics3['Population (2017)'] = demographics3['Population (2017)'].apply(lambda x: x.replace(',',''))

In [39]:
demographics3.head()

,Provincial Cities/Districts,Wards,Area (km2),Population (2017)
0,Ba Đình,14,9224,247100\n
1,Bắc Từ Liêm,13,4335,333300\n
2,Cầu Giấy,8,1204,266800\n
3,Đống Đa,21,996,420900\n
4,Hai Bà Trưng,20,1009,318000\n


In [40]:
# Add the "Density" column = Population / Area
demographics3["Density (pop/km2)"] = round(demographics3["Population (2017)"].astype(float) / demographics3["Area (km2)"].astype(float), 3)

In [41]:
from unidecode import unidecode
demographics3['Provincial Cities/Districts'] = demographics3['Provincial Cities/Districts'].apply(unidecode).str.strip()

ModuleNotFoundError: No module named 'unidecode'

In [42]:
demographics3['Provincial Cities/Districts'] = demographics3['Provincial Cities/Districts'].replace({'Ha DongHT':'Ha Dong'})

Get Housing Price in every district

In [43]:
source_housing_price = requests.get("https://mogi.vn/gia-nha-dat").text
soup = BeautifulSoup(source_housing_price, 'lxml')
table_housing_price = soup.find("div", class_="mt-table")

In [44]:
source_housing_price = requests.get("https://mogi.vn/gia-nha-dat").text
soup = BeautifulSoup(source_housing_price, 'lxml')
table_housing_price = soup.findAll("div", class_="mt-table")[1]

table_rows = table_housing_price.find_all("div", class_="mt-row")
housing_price = []
for tr in table_rows:
    district = tr.find("div", class_="mt-street").a.text
    medium_price = tr.find("div", class_="mt-vol text-right number").span.text
    row = [district, medium_price]
    housing_price.append(row)

In [45]:
housing_price

[['\nQuận Ba Đình\n', '167 triệu/m2'],
 ['\nQuận Cầu Giấy\n', '155 triệu/m2'],
 ['\nQuận Đống Đa\n', '162 triệu/m2'],
 ['\nQuận Hai Bà Trưng\n', '140 triệu/m2'],
 ['\nQuận Hoàn Kiếm\n', '442 triệu/m2'],
 ['\nQuận Hoàng Mai\n', '81,5 triệu/m2'],
 ['\nQuận Long Biên\n', '73,3 triệu/m2'],
 ['\nQuận Tây Hồ\n', '142 triệu/m2'],
 ['\nQuận Thanh Xuân\n', '115 triệu/m2'],
 ['\nQuận Hà Đông\n', '76,8 triệu/m2'],
 ['\nQuận Bắc Từ Liêm\n', '76,8 triệu/m2'],
 ['\nQuận Nam Từ Liêm\n', '85,2 triệu/m2'],
 ['\nHuyện Mê Linh\n', '12 triệu/m2'],
 ['\nHuyện Ba Vì\n', '2,4 triệu/m2'],
 ['\nHuyện Chương Mỹ\n', '10,8 triệu/m2'],
 ['\nHuyện Đan Phượng\n', '23,4 triệu/m2'],
 ['\nHuyện Hoài Đức\n', '40,8 triệu/m2'],
 ['\nHuyện Phúc Thọ\n', '4,8 triệu/m2'],
 ['\nHuyện Quốc Oai\n', '11,1 triệu/m2'],
 ['\nHuyện Thạch Thất\n', '11,4 triệu/m2'],
 ['\nHuyện Thanh Oai\n', '27,1 triệu/m2'],
 ['\nHuyện Thường Tín\n', '23 triệu/m2'],
 ['\nThị Xã Sơn Tây\n', '14,2 triệu/m2'],
 ['\nHuyện Đông Anh\n', '27,2 triệu/m2'],
 ['

In [46]:
house_price2 = pd.DataFrame(housing_price, 
                                columns=["District", "Average Housing Price (1M VND/m2)"])

In [47]:
house_price2 = house_price2.loc[:12].reset_index().drop("index", axis=1)

In [48]:
house_price2

,District,Average Housing Price (1M VND/m2)
0,\nQuận Ba Đình\n,167 triệu/m2
1,\nQuận Cầu Giấy\n,155 triệu/m2
2,\nQuận Đống Đa\n,162 triệu/m2
3,\nQuận Hai Bà Trưng\n,140 triệu/m2
4,\nQuận Hoàn Kiếm\n,442 triệu/m2
5,\nQuận Hoàng Mai\n,"81,5 triệu/m2"
6,\nQuận Long Biên\n,"73,3 triệu/m2"
7,\nQuận Tây Hồ\n,142 triệu/m2
8,\nQuận Thanh Xuân\n,115 triệu/m2
9,\nQuận Hà Đông\n,"76,8 triệu/m2"


In [49]:
# Remove the word "Quận"
house_price2["District"] = house_price2["District"].str.replace("\n", "").str.replace("Quận", "").str.strip()

In [50]:
house_price2

,District,Average Housing Price (1M VND/m2)
0,Ba Đình,167 triệu/m2
1,Cầu Giấy,155 triệu/m2
2,Đống Đa,162 triệu/m2
3,Hai Bà Trưng,140 triệu/m2
4,Hoàn Kiếm,442 triệu/m2
5,Hoàng Mai,"81,5 triệu/m2"
6,Long Biên,"73,3 triệu/m2"
7,Tây Hồ,142 triệu/m2
8,Thanh Xuân,115 triệu/m2
9,Hà Đông,"76,8 triệu/m2"


In [51]:
from unidecode import unidecode
# Remove Vietnamese accents
house_price2["District"] = house_price2["District"].apply(unidecode)

ModuleNotFoundError: No module named 'unidecode'

In [52]:
house_price2["Average Housing Price (1M VND/m2)"] = house_price2["Average Housing Price (1M VND/m2)"].str.replace("triệu", "").str.replace(",", ".").str.replace("/m2", "").str.strip()

In [53]:
house_price2

,District,Average Housing Price (1M VND/m2)
0,Ba Đình,167
1,Cầu Giấy,155
2,Đống Đa,162
3,Hai Bà Trưng,140
4,Hoàn Kiếm,442
5,Hoàng Mai,81.5
6,Long Biên,73.3
7,Tây Hồ,142
8,Thanh Xuân,115
9,Hà Đông,76.8


Data Merging now

In [54]:
data_hanoi = pd.merge(demographics3, house_price2, how = 'left', right_on = 'District', left_on = 'Provincial Cities/Districts').drop(columns={'District'})

In [55]:
data_hanoi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12 entries, 0 to 11
Data columns (total 6 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Provincial Cities/Districts        12 non-null     object 
 1   Wards                              12 non-null     object 
 2   Area (km2)                         12 non-null     object 
 3   Population (2017)                  12 non-null     object 
 4   Density (pop/km2)                  12 non-null     float64
 5   Average Housing Price (1M VND/m2)  11 non-null     object 
dtypes: float64(1), object(5)
memory usage: 672.0+ bytes


In [56]:
data_hanoi.head()

,Provincial Cities/Districts,Wards,Area (km2),Population (2017),Density (pop/km2),Average Housing Price (1M VND/m2)
0,Ba Đình,14,9224,247100\n,26.789,167
1,Bắc Từ Liêm,13,4335,333300\n,76.886,76.8
2,Cầu Giấy,8,1204,266800\n,221.595,155
3,Đống Đa,21,996,420900\n,422.590,162
4,Hai Bà Trưng,20,1009,318000\n,315.164,140


Coordinates of districts

In [64]:
# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim
# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize  

import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

# find the distances
from scipy.spatial.distance import cdist

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'geopy'

In [61]:
def find_coor(name):
#     address = name + "Hanoi City Vietnam"
    geolocator = Nominatim(user_agent="hnc")
    location = geolocator.geocode(name + " hanoi")
    latitude = location.latitude
    longitude = location.longitude
    return [latitude, longitude]

In [62]:
coords = [find_coor(dist + " District") for dist in data_hanoi["Provincial Cities/Districts"].iloc[:].tolist()]

df_coords = pd.DataFrame(coords, columns=["Latitude", "Longitude"])

data_hanoi["Latitude"] = df_coords["Latitude"]
data_hanoi["Longitude"] = df_coords["Longitude"]
data_hanoi

NameError: name 'Nominatim' is not defined

In [63]:
[hn_lat, hn_long] = find_coor("")
print('The geographical coordinates of Hanoi City are {}, {}.'.format(hn_lat, hn_long))

NameError: name 'Nominatim' is not defined

In [65]:
map_hn = folium.Map(location=[hn_lat, hn_long], zoom_start=12)
for lat, lng, dis in zip(data_hanoi['Latitude'], data_hanoi['Longitude'], data_hanoi['Provincial Cities/Districts']):
    label = '{}'.format(dis)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_hn)  

map_hn

NameError: name 'folium' is not defined

In [66]:
CLIENT_ID = '5ODBH5ASCVXDMJBQCFTK31SLJF24KTGO12IOJQZZLX0QDBBT'
CLIENT_SECRET = 'BNV2LP4YSWVNGDXP2E24OXSIMP5W2B1QIXULRV4ZHR000EJO'
VERSION = '20201109'

In [67]:
def getNearbyVenues(names, latitudes, longitudes, radius=1500, LIMIT=150):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [68]:
hn_venues = getNearbyVenues(names=data_hanoi['Provincial Cities/Districts'],
                             latitudes=data_hanoi['Latitude'],
                             longitudes=data_hanoi['Longitude']
                            )

KeyError: 'Latitude'

In [69]:
hn_venues.head()

NameError: name 'hn_venues' is not defined

In [70]:
#Count venues returned for each district
hn_venue_group = hn_venues.groupby('District').count().reset_index()
hn_venue_group

NameError: name 'hn_venues' is not defined